# Image & signal processing
Here, we'll demonstrate how you could implement visualization & processing techniques for biological time series and images using Python.

![](Data/leukaemia_smear.jpeg)

### By the end of this notebook, you'll be able to:
* Plot time series data with accurate timestamps
* Filter 2D and 3D data using convolution
* Display & manipulate images in Python
* Apply different types of filters to images

<hr>

First, let's get setup...

In [ ]:
# All of the lovely packages we need for today
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

# New package! Allows us to open URLs
import urllib.request

### Import and filter real sleep data

The data we'll import here is a real 30-seconds extract of slow-wave sleep from a young individual, collected by the Walker Lab at UC Berkeley. This data was collected at 100 Hz from channel 'F3'. This sampling frequency is fine for EEG data, but wouldn't be enough for high frequency spiking data. That kind of data is typically sampled at 40 **kilo**Hz.

In [ ]:
# URL of data to download
data_url = 'https://raphaelvallat.com/images/tutorials/bandpower/data.txt'

# Get the data and save it as "sleep_data.txt"
sleep_data, headers = urllib.request.urlretrieve(data_url, 'Data/sleep_data.txt')
sleep_data

<div class="alert alert-success">
    <b>Task</b>: Using a numpy function we've already learned, load your text file and assign it to "data". 

In [ ]:
# Load the .txt file


First, we'll plot the raw data. The data provides the information about voltage for the y axis. However, we need to also create a time vector, so that we can accurately label the x axis. The [`np.arange()`](https://numpy.org/doc/stable/reference/generated/numpy.arange.html) method is very useful for this. We need to create a vector that is the same length as our data, but where each number is in terms of **time**, not **samples**. To do so, **we'll divide our vector by the sampling rate.**

<div class="alert alert-success">

**Task**: Create a `time_vector` variable so that your data is plotted in seconds, not samples.

</div>

In [ ]:
# Define sampling frequency, num_samples, and time vector
sampling_freq = 100 # sampling frequency, in Hz
timestep = ...
time_vector = ...

<div class="alert alert-success">

**Task**: Plot your signal! You only need to add a line of code where it says `# ADD CODE HERE`

</div>

In [ ]:
# Plot the signal
fig, ax = plt.subplots(figsize=(12, 4))

# ADD CODE HERE

plt.xlabel('Time (seconds)')
plt.ylabel('Voltage')
plt.title('N3 sleep EEG data (F3)')
plt.show()

### Filtering a signal with convolution
Let's start with a simple step signal (left) which we'll add some noise to (right).

In [ ]:
step_signal = np.zeros(100) # Create a 1D array of all 0s
step_signal[50:] = 1        # Set 50: to 1

fig, ax = plt.subplots(1,2,figsize=(10,4))
ax[0].plot(step_signal)

# Create random signal
np.random.seed(0) # Set the "seed" for randomness

noisy_signal = (step_signal + np.random.normal(0, 0.35, step_signal.shape))
ax[1].plot(noisy_signal)

plt.show()

If our goal is to cover something that looks more like the image on the left, we can **smooth** the signal. In the first line below, we'll take each point and average it by one datapoint to the left and one datapoint to the right, discarding the ends of the signal, where we can't do this operation.

In [ ]:
# Take the mean of neighboring datapoints
smooth_signal = (noisy_signal[:-1] + noisy_signal[1:]) / 2.0

plt.plot(smooth_signal)
plt.show()

Above, we're only averaging by one datapoint on each side. What if we average by 3 datapoints? Below, we'll do the following:
1. Create an output array called `smooth_signal3`, of the same length as noisy_signal.
2. At each element in `smooth_signal3` starting at point 1, and ending at point -2, place the average of the sum of: 1/3 of the element to the left of it in noisy_signal, 1/3 of the element at the same position, and 1/3 of the element to the right.
3. Discard the leftmost and rightmost elements.

In [ ]:
# Generate our signal
smooth_signal3 = (noisy_signal[:-2] + noisy_signal[1:-1] + noisy_signal[2:]) / 3

plt.plot(smooth_signal, label='mean of 2')
plt.plot(smooth_signal3, label='mean of 3')
plt.legend(loc='upper left')
plt.show()

If we want to continue this, the first line of our code is going to continue to get longer, and there isn't a straightforward way to make this flexible for how many points we want to average. Thankfully, there's a name for what we're doing: **convolving**. This same concept, nearest-neighbor averages, can be expressed as a **convolution with an averaging kernel**.

**Convolution** is the process of adding each element of the image to its local neighbors, weighted by the kernel. 

In [ ]:
# Same as above, using a convolution kernel
# Neighboring pixels multiplied by 1/3 and summed

# Create the "kernel". For 3, this creates an array that looks like [0.333,0.333,0.333]
mean_kernel3 = np.full((3,), 1/3)

# Smooth the signal by convolving it
smooth_signal3p = np.convolve(noisy_signal, mean_kernel3, mode='valid')

plt.plot(smooth_signal3p)
plt.show()

In [ ]:
# Check that they're equal!
print('smooth_signal3 and smooth_signal3p are equal:',
      np.allclose(smooth_signal3, smooth_signal3p))

<div class="alert alert-success">

**Tasks**:
    
1. What happens if we convolve with a kernel of size 10? Also, do these arrays have the same # of data points? If not, why?
2. Smooth our sleep recording with different kernel sizes and inspect.
   
</div>

In [ ]:
# Try a kernel of size 10 here


In [ ]:
# Smooth the sleep recording we imported above!


## Images in Python
Below, we'll use `mpimg.imread()` to read in an image file and [`plt.imshow()`](https://matplotlib.org/devdocs/api/_as_gen/matplotlib.pyplot.imshow.html) to show [our image](https://en.wikipedia.org/wiki/Cancer_cell#/media/File:Acute_lymphoblastic_leukaemia_smear.jpg).

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg # New module to read in images

cells = mpimg.imread('Data/leukaemia_smear.jpeg')
plt.axis('off') # Turn axis ticks & labels off
plt.imshow(cells)
plt.show()

We can work with our `cells` object just like we would any other object. Let's check its shape. 

In [ ]:
# Check cells shape
cells.shape

If the first and second value are the size, what is the third value?
<div class="alert alert-success">

**Task**: Below, use only the first values in the third dimension to generate a different version of our image that is only based on the "R" luminance. Call this `cells_gray`.

</div>

In [ ]:
cells_gray = ...

By default, matplotlib uses the viridis color map to plot a sequence of values (you can see the luminance at each pixel by showing the colorbar with `plt.colorbar()`).
<div class="alert alert-success">
    
 **Task**. Change your image to a different colormamp using the `cmap` argument in `plt.imshow`. There are many [colormaps](https://matplotlib.org/stable/tutorials/colors/colormaps.html) -- try some below!
    
</div>

In [ ]:
# Try your color maps here


### Filtering images to smooth them
In our signal processing tutorial above, we talked about convolving a signal with an averaging kernel to smooth the signal. We can apply similar logic to images. Let's start with a very simple image.

In [7]:
bright_square = np.zeros((7, 7), dtype=float)
bright_square[2:5, 2:5] = 1
bright_square

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

This gives the values above, and looks like this!

In [ ]:
plt.imshow(bright_square,cmap='gray')
plt.axis('off')
plt.show()

For our first example of a filter, consider the following filtering array, which we’ll call a “mean kernel”. For each pixel, a kernel defines which neighboring pixels to consider when filtering, and how much to weight those pixels.

In [ ]:
mean_kernel = np.full((3, 3), 1/9)

print(mean_kernel)

Now, let’s take our mean kernel and apply it to every pixel of the image.

Applying a (linear) filter essentially means:

1. Center a kernel on a pixel

2. Multiply the pixels under that kernel by the values in the kernel

3. Sum all the those results

4. Replace the center pixel with the summed result

This process is known as **convolution** (the same process we applied to a signal, but now in 3D!)

In [ ]:
# Import image processing toolbox
import scipy.ndimage as ndi

# Set precision
%precision 2 

print(ndi.correlate(bright_square, mean_kernel))

In [ ]:
smooth_square = ndi.correlate(bright_square, mean_kernel)
plt.imshow(smooth_square)
plt.show()

Below we'll show a short demo of what this looks like. If you're curious how this runs, you can dig into the `convolution_demo.py` file.

**Tip for a6**: Note how we are importing and using this custom module!

In [10]:
from Demos import convolution_demo

convolution_demo.mean_filter_interactive_demo(bright_square);

interactive(children=(IntSlider(value=0, description='i_step', max=48), Output()), _dom_classes=('widget-inter…

This kind of mean filter is a bit brute. Typically, we'd use a **Gaussian filter**, where the amount of filtering depends on the distance from the center point. Thankfully, we can do that using the filters module:

In [ ]:
from skimage import filters

gaussian_cells = filters.gaussian(cells_gray,3)

plt.imshow(gaussian_cells)
plt.show()

### Edge filtering
In biology, we often want to filter edges to identify different features within cells, or cells within a piece of tissue. To do so, we'll often use some sort of edge filtering. To give you an intuition for how edge filtering works, first consider taking our step function and convolving it with a filter that is [-1 0 1].

**Note**: For technical signal processing reasons, convolutions actually occur “back to front” between the input array and the kernel. Correlations occur in the signal order, so we’ll use correlate from now on.

In [ ]:
# Recreate the step signal
step_signal = np.zeros(100)
step_signal[50:] = 1

# Convolve it with a [-1 0 1] filter
edge_filter = np.correlate(step_signal, np.array([-1, 0, 1]),mode='valid')

plt.plot(step_signal, label='signal')
plt.plot(edge_filter, linestyle='dashed', label='correlated')
plt.legend(loc='upper left')
plt.show()

Whenever neighboring values are close, the filter response is close to 0. Right at the boundary of a step, we’re subtracting a small value from a large value and and get a spike in the response. This spike “identifies” our edge.

Now, let's apply this to images. Below, we'll just apply one filter, in the vertical direction.

In [ ]:
vertical_kernel = np.array([
    [-1],
    [ 0],
    [ 1],
])

horizontal_kernel = np.array([[-1,0,1]]) # Create a horizontal kernel

gradient_vertical = ndi.correlate(cells_gray.astype(float),vertical_kernel)
gradient_horizontal = ndi.correlate(cells_gray.astype(float),horizontal_kernel)

fig, ax = plt.subplots(1,2,figsize=(10,5))
ax[0].axis('off')
ax[1].axis('off')
ax[0].imshow(gradient_vertical,cmap='gray')
ax[1].imshow(gradient_horizontal,cmap='gray')

plt.show()

As you can see, applying a filter in the vertical direction helps us identify horizontal edges.
<div class="alert alert-success">

**Task**: Change the filter to a horizontal one and observe how this changes the edge detection.
</div>
    
Thankfully, there's a commonly used filter, the **Sobel** edge filter, that does this for us. First, let's appreciate what it looks like for our bright square.

In [ ]:
plt.imshow(filters.sobel(bright_square))
plt.show()

Now, let's apply it to our cells! 
<div class="alert alert-success">
    
**Task**: Create a figure where the left subplot is a filter of our raw cell image. On the right subplot, apply it to the Gaussian filtered image.
    
</div>

In [ ]:
# Create your figure here


Note how even just a little bit of preprocessing with a Gaussian filter changes the edge detection!

### Final fun challenge
<div class="alert alert-success">

**Task**: Using numpy arrays, can you write your initials?

</div>

In [ ]:
# Write your initials here


<hr>

## About this notebook
Some of the code in this notebook was adapted from [this tutorial](https://raphaelvallat.com/bandpower.html) by Raphael Vallat, [these tutorials](https://github.com/voytekresearch/Tutorials) from Torben Noto, _Neural Data Science_ by Pascal Wallisch, and [this tutorial](https://jni.github.io/i2k-skimage-napari/lectures/1_image_filters.html#local-filtering).